<a href="https://colab.research.google.com/github/NathanyApSalles/analysis_foodtech/blob/main/Case_Tecnico_DataAnalyst_Ifood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bases

In [54]:
import requests
import tarfile
import os
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession.builder.getOrCreate()

In [55]:
def read_file(url: str, local_path: str, type_file: str) -> DataFrame:
  """Função para ler arquivo e retornar um Dataframe."""
  if not os.path.exists(local_path):
    response = requests.get(url)
    with open(local_path, "wb") as f:
        f.write(response.content)
  if type_file == "json":
    return spark.read.json(local_path, multiLine=False)
  elif type_file == "csv":
    return spark.read.option("header", "true").csv(local_path)
  elif type_file == "tar":

    arquivos_extraidos = "/tmp/ab_test_ref"

    os.makedirs(arquivos_extraidos, exist_ok=True)

    with tarfile.open(local_path, "r:gz") as tar:
        tar.extractall(path=arquivos_extraidos)

    filename = ""
    for root, dirs, files in os.walk(arquivos_extraidos):
        for filename in files:
            print(filename) # print para visualizar todos os arquivos exraídos
    if ".csv" in filename: # se houver algum arquivo csv, junte todos os arquivos deste tipo no dataframe
      return spark.read.option("header", "true").csv(arquivos_extraidos + "/*.csv")
    else: # se houver algum arquivo json, junte todos os arquivos deste tipo no dataframe
      return spark.read.json(arquivos_extraidos + "/*.json")
      #pode acontecer de ter arquivos de diferentes tipos misturados, mas para este estudo vamos assumir que todos são do mesmo tipo


In [56]:
url_pedidos = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
url_usuarios = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
url_merchants = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
url_test_ab = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"

local_path_pedidos = "/tmp/order.json.gz"
local_path_usuarios = "/tmp/consumer.csv.gz"
local_path_merchants = "/tmp/restaurant.csv.gz"
local_path_teste_ab = "/tmp/ab_test_ref.tar.gz"

pedidos = read_file(url_pedidos, local_path_pedidos, "json")
usuarios = read_file(url_usuarios, local_path_usuarios, "csv")
merchants = read_file(url_merchants, local_path_merchants, "csv")
teste_ab = read_file(url_test_ab, local_path_teste_ab, "tar")

ab_test_ref.csv
._ab_test_ref.csv


In [57]:
pedidos.show(5)

+-----------+--------------------+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+--------------------+--------------------+-----------------+------------------+-----------------+--------------------+--------------------+---------------+--------------------+------------------+---------------+
|        cpf|         customer_id|customer_name|delivery_address_city|delivery_address_country|delivery_address_district|delivery_address_external_id|delivery_address_latitude|delivery_address_longitude|delivery_address_state|delivery_address_zip_code|               items|         merchant_id|merchant_latitude|merchant_longitude|merchant_timezone|    order_created_at|            order_id|order_scheduled|order_scheduled_date|order_total_amount|origin_platform|
+-----------+--------------------+-------------+---------------------+--

In [58]:
usuarios.show(5)

+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+
|         customer_id|language|          created_at|active|customer_name|customer_phone_area|customer_phone_number|
+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+
|e8cc60860e09c0bb1...|   pt-br|2018-04-05T14:49:...|  true|         NUNO|                 46|            816135924|
|a2834a38a9876cf74...|   pt-br|2018-01-14T21:40:...|  true|     ADRIELLY|                 59|            231330577|
|41e1051728eba1334...|   pt-br|2018-01-07T03:47:...|  true|        PAULA|                 62|            347597883|
|8e7c1dcb64edf95c9...|   pt-br|2018-01-10T22:17:...|  true|       HELTON|                 13|            719366842|
|7823d4cf4150c5dae...|   pt-br|2018-04-06T00:16:...|  true|       WENDER|                 76|            543232158|
+--------------------+--------+--------------------+------+-------------

In [59]:
merchants.show(5)

+--------------------+--------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|                  id|          created_at|enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|merchant_zip_code| merchant_city|merchant_state|merchant_country|
+--------------------+--------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|d19ff6fca6288939b...|2017-01-23T12:52:...|  false|          3|          60.0|           0|           50|               30.0|            14025|RIBEIRAO PRETO|            SP|              BR|
|631df0985fdbbaf27...|2017-01-20T13:14:...|   true|          3|          60.0|           0|            0|               30.0|            50180|     SAO PAULO|            SP|              BR|
|135c5c4ae4c1ec1fd...|2017-01-23T12:46:...|  

In [60]:
teste_ab.show(5, truncate=False)

+----------------------------------------------------------------+---------+
|customer_id                                                     |is_target|
+----------------------------------------------------------------+---------+
|755e1fa18f25caec5edffb188b13fd844b2af8cf5adedcf77c028f36cb9382ea|target   |
|b821aa8372b8e5b82cdc283742757df8c45eecdd72adf411716e710525d4edf1|control  |
|d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc007f5297541ec40b63b|control  |
|6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c8b535718bbc651ab02|target   |
|dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f5e3763fa657c23a58d|control  |
+----------------------------------------------------------------+---------+
only showing top 5 rows



In [61]:
from pyspark.sql.functions import col, broadcast, when, count, isnan, count_distinct

# Entendendo os dados

In [62]:
def df_info(df: DataFrame) -> None:
  """Função ."""
  # validando o tipo das colunas
  df.printSchema()

  # validando o tamanho da base
  print(f"Qtd de linhas: {df.count()}")

  # validando valores não válidos
  print("\nValores inválidos")
  # df.select([count(when(isnan(col(c)), c)).alias(c) for c in df.columns]).show()

  print("\nValores nulos")
  # validando valores nulos
  df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()


In [63]:
df_info(usuarios)

root
 |-- customer_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- active: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_phone_area: string (nullable = true)
 |-- customer_phone_number: string (nullable = true)

Qtd de linhas: 806156

Valores inválidos

Valores nulos
+-----------+--------+----------+------+-------------+-------------------+---------------------+
|customer_id|language|created_at|active|customer_name|customer_phone_area|customer_phone_number|
+-----------+--------+----------+------+-------------+-------------------+---------------------+
|          0|       0|         0|     0|            0|                  0|                    0|
+-----------+--------+----------+------+-------------+-------------------+---------------------+



In [ ]:
df_info(pedidos)

root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: string (nullable = true)
 |-- merchant_longitude: string (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_scheduled_date: string (nullable = true)
 |--

In [ ]:
# removendo os usuários nulos da base
#na base de pedidos temos um número pequeno de pedidos sem customer_id atribuído,
#porém como o objetivo da análise é validar os resultados do teste a/b, estes pedidos serão removidos visto que
#não participaram do teste a/b
pedidos_validos_ab = pedidos.where(col("customer_id").isNotNull())
df_info(pedidos_validos_ab)

In [ ]:
df_info(merchants)

In [ ]:
df_info(teste_ab)

# EDA

In [ ]:
pedidos_final = pedidos_validos_ab.join(broadcast(teste_ab), ['customer_id'], "left")
pedidos_final.show(5)

In [ ]:
# removendo os pedidos em que os usuários não participaram do teste a/b

pedidos_final.where(col("is_target").isNotNull()).groupBy(col("is_target")).agg(count_distinct(col("order_id")).alias("distinct_orders"),
                                      count_distinct(col("customer_id")).alias("distinct_customers"),
                                      count_distinct(col("merchant_id")).alias("distinct_merchants")


                                      ).show()

In [ ]:
pedidos.select("order_id").where(col("is_target").isNull()).show()

In [ ]:
usuarios.where(isnan(col("customer_name"))).show()

- tarquet x control

    - qtd de pedidos
    - valor total da compra
    - ticket médio
    - tempo entre compras
    - recorrencia
    - tem diferença entre usuários ativos ou não?
    - expansão geográfica: qtd de estabelecimentos diferentes que compraram
    - restaurantes que mais venderam
    - região que mais vendeu
    - horário das compras
    - qtd de produtos adquiridos

In [ ]:
pedidos.join(broadcast(teste_ab), ['customer_id'], "left").show(5)

In [ ]:
pedidos.where(col("customer_id") == "d2c9808453a83b65977dc9f10adda60e3edd3fff5959b15efe9d865baecefdec").show(truncate=False)

In [ ]:
usuarios.where(col("active") == False).show(truncate=False)